In [14]:
from interpret.blackbox import LimeTabular
from interpret import show
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

## Zadanie

Proszę przygotować rozwiązanie (program, funkcję, polecenie) w języku Python, które pozwoli na zweryfikowanie istotności predykatorów zmiennej "survived" zbioru Titanic. Należy zbudować klasyfikator wykorzystując jeden z algorytmów typu blackbox a następnie wybrać 4 różne przypadki pasażerów i sprawdzić, które ze zmiennych miały największy wpływ na przypisanie przez klasyfikator etykiety danej klasy.

**Wyniki należy zinterpretować.**

In [15]:
df = pd.read_csv("titanic.csv", index_col='Unnamed: 0')
df.head()

,gender,age,class,embarked,fare,sibsp,parch,survived
0,1,42.0,2,3,7.11,0,0,0
1,1,13.0,2,3,20.05,0,2,0
2,1,16.0,2,3,20.05,1,1,0
3,0,39.0,2,3,20.05,1,1,1
4,0,16.0,2,3,7.13,0,0,1


In [16]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [18]:
pca = PCA()

In [19]:
rf = RandomForestClassifier(n_estimators=100)
lr = LogisticRegression()

In [20]:
# blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model = Pipeline([('lr', lr)])
blackbox_model.fit(X_train.values, y_train)

Pipeline(steps=[('lr', LogisticRegression())])

In [21]:
blackbox_model.score(X_test, y_test)

C:\Users\mateu\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but LogisticRegression was fitted without feature names



0.7895927601809954

In [22]:
lime_inter = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=100)

In [23]:
display(X_test.iloc[55:59, :])
print('Survived:')
display(y_test.iloc[55:59])

,gender,age,class,embarked,fare,sibsp,parch
840,1,4.0,2,1,15.0411,1,1
220,0,32.0,0,3,53.0200,1,0
1903,1,39.0,6,3,0.0000,0,0
562,1,42.0,0,3,52.0000,1,0


Survived:


840     1
220     1
1903    0
562     0
Name: survived, dtype: int64

In [24]:
inter_lokal = lime_inter.explain_local(X_test.iloc[55:59, :], y_test.iloc[55:59], name='lime')
show(inter_lokal)

W dwóch przedstawionych przypadkach najlepszą zmienną wykazała się być zmienna "fare", a w pozostałych dwóch zmienna "sibsp". Zmienna fare określa wielkość opłaty za rejs, a zmienna sibsp określa liczbę rodzeństwa lub małżonków na pokładzie. <br><br>
W trzech przypadkach prawdopodobieństwo predykcji nie przekracza 0.3, co jest dość słabym wynikiem. Tylko jeden przypadek ma prawdopodobieństwo predykcji na dość wysokim poziomie - 0.729.